% title: Jupyter Notebooks and Python
% author: Jon Wheeler
% date: 2022-03-03

# Jupyter Notebooks & Python

## Abstract

Workshop tutorial with a hands-on demonstration of Markdown, Jupyter Notebooks (Python) and R Markdown (R). 

*keywords*: markdown, jupyter, python, literate computing